In [ ]:
import torch
from tqdm import tqdm
# 加载YOLOv5模型
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

In [ ]:

# 定义检测的类（只检测车辆相关的类别）
classes_to_detect = ['bicycle', 'car', 'motorbike', 'bus', 'truck']


In [ ]:
from tools.video import get_video_paths

video_paths = get_video_paths("./dataset/train")

In [ ]:
import time
import csv
import os
import cv2
from tqdm import tqdm

# 初始化变量
all_output_data = []  # 存储所有视频的运行数据

for path in tqdm(video_paths):
    file_folder = path.split('/')[-2]
    full_file_folder = str('/').join(path.split('/')[:-1])
    
    # 打开视频文件
    filename = os.path.basename(path)
    video_name = os.path.splitext(filename)[0]
    cap = cv2.VideoCapture(path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # 获取视频帧率

    # 获取视频的帧数
    all_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"视频总帧数: {all_frame_count}")

    # 定义用于计算流量的线的位置
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    line_position = frame_height // 2  # 根据需要调整位置

    # 初始化变量
    frame_count = 0
    output_data = []

    # 定义处理间隔（每25帧处理一次）
    process_frame_interval = 25  # 每25帧处理一次

    # 定义输出间隔（以处理的帧数为单位）
    output_frame_interval = 1  # 每处理25帧输出一次

    processed_frame_count = 0  # 处理过的帧数计数器

    # 记录开始时间
    start_time = time.time()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1

        # 仅每隔指定的帧数处理一次
        if frame_count % process_frame_interval != 0:
            continue

        processed_frame_count += 1  # 增加处理过的帧数计数器

        # 使用YOLOv5模型进行推理
        results = model(frame)
        detections = results.xyxy[0].cpu().numpy()  # [x1, y1, x2, y2, confidence, class_id]

        boxes = []
        confidences = []
        class_ids = []

        for det in detections:
            confidence = det[4]
            class_id = int(det[5])
            if confidence > 0.3:  # 使用置信度阈值
                class_name = model.names[class_id]
                if class_name in classes_to_detect:
                    x1, y1, x2, y2 = int(det[0]), int(det[1]), int(det[2]), int(det[3])
                    boxes.append([x1, y1, x2 - x1, y2 - y1])
                    confidences.append(confidence)
                    class_ids.append(class_id)

        # 计算密度：当前帧中的车辆数量
        density = len(boxes)

        # 计算流量：判断位于指定线位置的车辆数量
        flow = 0
        for box in boxes:
            x1, y1, x2, y2 = box
            # 检查车辆是否与线相交
            if y1 <= line_position <= y2:
                flow += 1

        # 每隔一定的处理帧数输出一次数据
        if processed_frame_count % output_frame_interval == 0:
            output = {
                'Frame': frame_count,
                'Flow': flow,
                'Density': density
            }
            print(f"视频总帧数: {all_frame_count}")
            print(output)
            output_data.append(output)

    # 记录结束时间
    end_time = time.time()

    # 计算总处理时间
    total_time = end_time - start_time

    # 计算处理帧率（每秒处理帧数）
    process_fps = processed_frame_count / total_time

    # 保存每个视频的统计数据
    video_output = {
        'Video Name': video_name,
        'Total Frames': all_frame_count,
        'Processed Frames': processed_frame_count,
        'Total Time (s)': total_time,
        'Process FPS': process_fps
    }
    all_output_data.append(video_output)

    cap.release()

# 保存所有视频的处理数据到CSV文件
output_csv_path = "yolov5_runtime_stats.csv"
with open(output_csv_path, mode='w', newline='') as csv_file:
    fieldnames = ['Video Name', 'Total Frames', 'Processed Frames', 'Total Time (s)', 'Process FPS']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for data in all_output_data:
        writer.writerow(data)

print(f"所有视频处理数据已保存到 {output_csv_path}")
